# Delta Lake, Apache Iceberg & DuckLake

## Format-Überblick (kurz)
**Delta Lake**
- Offenes Format auf Parquet-Basis mit Transaktionslog (`_delta_log`).
- Stärken: Spark-Ökosystem, Time‑Travel, MERGE/UPSERT, breites Tooling.
- Schwächen: Spark-Abhängigkeit für viele Features; Log-Vacuum beachten.

**Apache Iceberg**
- Tabellenformat mit **Snapshots**, versteckter Partitionierung und flexibler Schema‑Evolution.
- Stärken: Engine‑neutral (Spark, Flink, Trino, DuckDB), performantes Metadata‑Layout.
- Schwächen: Katalogverwaltung (REST/Nessie/Glue/etc.) & Setup‑Varianten können komplex sein.

**DuckLake**
- Neues offenes Lakehouse‑Format (Metadaten in relationaler DB, z. B. SQLite; Daten als Parquet).
- Stärken: einfache lokale Kataloge, schnelle Demos, Snapshot‑APIs; Engine‑agnostisch via DuckDB.
- Schwächen: junges Ökosystem; weniger Integrationen als Iceberg/Delta (Stand: 2025).

## Gewählte Datenquelle
- NYC Green Taxi (01/2019, Parquet) → `https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-01.parquet`
- Für die Demos wird eine **Teilmenge** benutzt (Limit ~5k Zeilen), um Rechenzeit in Colab kurz zu halten.

In [ ]:
DATA_URL = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-01.parquet'
print('Datenquelle:', DATA_URL)

Datenquelle: https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-01.parquet


## Delta Lake


In [2]:
!apt-get install -qq openjdk-11-jdk-headless > /dev/null
!pip install -q pyspark==3.5.1 delta-spark==3.2.0

In [3]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = (
    SparkSession.builder
    .appName("DeltaSchemaEvolutionDemo")
    .config(
        "spark.sql.extensions",
        "io.delta.sql.DeltaSparkSessionExtension"
    )
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog"
    )
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

print("Spark:", spark.version)
print("Extensions:", spark.conf.get("spark.sql.extensions"))

Spark: 3.5.1
Extensions: io.delta.sql.DeltaSparkSessionExtension


In [4]:
base_path = "/content/delta/events_demo"

events_v1 = spark.createDataFrame(
    [
        (1, "signup"),
        (2, "purchase"),
        (3, "signup"),
    ],
    ["user_id", "event_type"]
)

(
    events_v1.write
        .format("delta")
        .mode("overwrite")
        .save(base_path)
)

df = spark.read.format("delta").load(base_path)
df.printSchema()
df.show()

root
 |-- user_id: long (nullable = true)
 |-- event_type: string (nullable = true)

+-------+----------+
|user_id|event_type|
+-------+----------+
|      2|  purchase|
|      3|    signup|
|      1|    signup|
+-------+----------+



In [5]:
# Schema ändern
events_v2 = spark.createDataFrame(
    [
        (4, "signup",   "mobile"),
        (5, "purchase", "web"),
    ],
    ["user_id", "event_type", "device_type"]
)

(
    events_v2.write
        .format("delta")
        .mode("append")
        .option("mergeSchema", "true")  # Schema-Evolution
        .save(base_path)
)

df2 = spark.read.format("delta").load(base_path)
df2.printSchema()
df2.orderBy("user_id").show()

root
 |-- user_id: long (nullable = true)
 |-- event_type: string (nullable = true)
 |-- device_type: string (nullable = true)

+-------+----------+-----------+
|user_id|event_type|device_type|
+-------+----------+-----------+
|      1|    signup|       NULL|
|      2|  purchase|       NULL|
|      3|    signup|       NULL|
|      4|    signup|     mobile|
|      5|  purchase|        web|
+-------+----------+-----------+



In [6]:
# Zugriff mittels SQL
base_path = "/content/delta/events_demo"  # wie beim Schreiben

spark.sql(f"""
  CREATE TABLE IF NOT EXISTS events_demo
  USING DELTA
  LOCATION '{base_path}'
""")

spark.sql("SELECT * FROM events_demo ORDER BY user_id").show()

+-------+----------+-----------+
|user_id|event_type|device_type|
+-------+----------+-----------+
|      1|    signup|       NULL|
|      2|  purchase|       NULL|
|      3|    signup|       NULL|
|      4|    signup|     mobile|
|      5|  purchase|        web|
+-------+----------+-----------+



In [7]:
# Alternativ DDL erweitern
spark.sql("""
  ALTER TABLE events_demo
  ADD COLUMNS (event_date DATE)
""")

spark.sql("DESCRIBE TABLE events_demo").show(truncate=False)
spark.sql("SELECT * FROM events_demo ORDER BY user_id").show()

+-----------+---------+-------+
|col_name   |data_type|comment|
+-----------+---------+-------+
|user_id    |bigint   |NULL   |
|event_type |string   |NULL   |
|device_type|string   |NULL   |
|event_date |date     |NULL   |
+-----------+---------+-------+

+-------+----------+-----------+----------+
|user_id|event_type|device_type|event_date|
+-------+----------+-----------+----------+
|      1|    signup|       NULL|      NULL|
|      2|  purchase|       NULL|      NULL|
|      3|    signup|       NULL|      NULL|
|      4|    signup|     mobile|      NULL|
|      5|  purchase|        web|      NULL|
+-------+----------+-----------+----------+



# Apache Iceberg Copy on write vs Merge on read

In [ ]:
#!apt-get install -qq openjdk-11-jdk-headless > /dev/null
#!pip install -q pyspark==3.5.1

# Sitzung neu starten!

In [1]:
DATA_URL = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-01.parquet'
print('Datenquelle:', DATA_URL)

Datenquelle: https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-01.parquet


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .appName("IcebergLocalDevelopment") \
  .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2') \
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
  .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
  .config("spark.sql.catalog.local.type", "hadoop") \
  .config("spark.sql.catalog.local.warehouse", "spark-warehouse/iceberg") \
  .getOrCreate()
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [3]:
import pandas as pd

pdf = pd.read_parquet(
    DATA_URL,
    engine="pyarrow",
    columns=[
        "lpep_pickup_datetime",
        "lpep_dropoff_datetime",
        "passenger_count",
        "trip_distance",
        "total_amount",
    ],
).head(5000)

# pandas → Spark-DataFrame
trips_df = spark.createDataFrame(pdf)

In [4]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS local.schema")

DataFrame[]

In [5]:
(
    trips_df.writeTo("local.schema.trips_cow")
          .tableProperty("format-version", "2")
          .tableProperty("write.update.mode", "copy-on-write")
          .createOrReplace()
)

In [6]:
(
    trips_df.writeTo("local.schema.trips_mor")
          .tableProperty("format-version", "2")
          .tableProperty("write.update.mode", "merge-on-read")
          .createOrReplace()
)

In [7]:
spark.sql("select * from local.schema.trips_mor").show()

+--------------------+---------------------+---------------+-------------+------------+
|lpep_pickup_datetime|lpep_dropoff_datetime|passenger_count|trip_distance|total_amount|
+--------------------+---------------------+---------------+-------------+------------+
| 2018-12-21 15:17:29|  2018-12-21 15:18:57|            5.0|          0.0|         4.3|
| 2019-01-01 00:10:16|  2019-01-01 00:16:32|            2.0|         0.86|         7.3|
| 2019-01-01 00:27:11|  2019-01-01 00:31:38|            2.0|         0.66|         5.8|
| 2019-01-01 00:46:20|  2019-01-01 01:04:54|            2.0|         2.68|       19.71|
| 2019-01-01 00:19:06|  2019-01-01 00:39:43|            1.0|         4.53|        19.3|
| 2019-01-01 00:12:35|  2019-01-01 00:19:09|            1.0|         1.05|         7.8|
| 2019-01-01 00:47:55|  2019-01-01 01:00:01|            1.0|         3.77|        14.8|
| 2019-01-01 00:12:47|  2019-01-01 00:30:50|            1.0|          4.1|        17.3|
| 2019-01-01 00:16:23|  2019-01-

Bei COW-Table wird bei einem UPDATE die betroffene Datei komplett neu geschrieben.
Bei MOR-Table wird nichts „überschrieben“, sondern es werden neue Daten + Delete-Datei angelegt und die alten Dateien bleiben erhalten.

In [10]:
spark.sql("UPDATE local.schema.trips_cow SET passenger_count = passenger_count + 1")
spark.sql("UPDATE local.schema.trips_mor SET passenger_count = passenger_count + 1")

DataFrame[]

In [11]:
#spark.sql("SELECT * FROM local.schema.trips_cow").show()

# Welche Dateien gehören zu welchen Snapshots?
spark.sql("""
  SELECT
    content,         -- 0 = Data, 1 = Position-Delete, 2 = Equality-Delete
    file_format,
    file_path,
    record_count
  FROM local.schema.trips_cow.files
  ORDER BY content, file_path
""").show(truncate=False)


+-------+-----------+----------------------------------------------------------------------------------------------------------+------------+
|content|file_format|file_path                                                                                                 |record_count|
+-------+-----------+----------------------------------------------------------------------------------------------------------+------------+
|0      |PARQUET    |spark-warehouse/iceberg/schema/trips_cow/data/00000-8-b4eeac2c-57fa-41f5-9567-4818fd707d9e-0-00001.parquet|5000        |
+-------+-----------+----------------------------------------------------------------------------------------------------------+------------+



In [12]:
#spark.sql("SELECT * FROM local.schema.trips_mor").show()

spark.sql("""
  SELECT
    content,         -- 0 = Data, 1 = Position-Delete, 2 = Equality-Delete
    file_format,
    file_path,
    record_count
  FROM local.schema.trips_mor.files
  ORDER BY content, file_path
""").show(truncate=False)

+-------+-----------+-----------------------------------------------------------------------------------------------------------------+------------+
|content|file_format|file_path                                                                                                        |record_count|
+-------+-----------+-----------------------------------------------------------------------------------------------------------------+------------+
|0      |PARQUET    |spark-warehouse/iceberg/schema/trips_mor/data/00000-10-15cf5e6d-9035-491b-98f7-38731f25179a-00001.parquet        |5000        |
|0      |PARQUET    |spark-warehouse/iceberg/schema/trips_mor/data/00000-2-84080325-ed0b-422a-afee-6b6df39bb8c0-0-00001.parquet       |2048        |
|0      |PARQUET    |spark-warehouse/iceberg/schema/trips_mor/data/00001-3-84080325-ed0b-422a-afee-6b6df39bb8c0-0-00001.parquet       |2952        |
|1      |PARQUET    |spark-warehouse/iceberg/schema/trips_mor/data/00000-10-15cf5e6d-9035-491b-98f7-38731f

Die beiden Dateien mit
2048 und 2952 Zeilen (zusammen 2048 + 2952 = 5000) sind die ursprünglichen Datenfiles vor dem UPDATE.

Das File mit record_count = 5000 und content = 0 ist die neue Daten-Datei mit den geänderten passenger_count-Werten.

Das deletes.parquet mit content = 1 und record_count = 5000 ist eine Position-Delete-Datei:

sie enthält 5000 Einträge, die auf alle ursprünglichen 5000 Zeilen in den zwei alten Dateien verweisen

damit werden diese alten Zeilen logisch gelöscht.

In [13]:
spark.sql("UPDATE local.schema.trips_mor SET passenger_count = passenger_count + 1 WHERE total_amount > 240")

DataFrame[]

In [14]:
spark.sql("""
  SELECT
    content,         -- 0 = Data, 1 = Position-Delete, 2 = Equality-Delete
    file_format,
    file_path,
    record_count
  FROM local.schema.trips_mor.files
  ORDER BY content, file_path
""").show(truncate=False)

+-------+-----------+-----------------------------------------------------------------------------------------------------------------+------------+
|content|file_format|file_path                                                                                                        |record_count|
+-------+-----------+-----------------------------------------------------------------------------------------------------------------+------------+
|0      |PARQUET    |spark-warehouse/iceberg/schema/trips_mor/data/00000-10-15cf5e6d-9035-491b-98f7-38731f25179a-00001.parquet        |5000        |
|0      |PARQUET    |spark-warehouse/iceberg/schema/trips_mor/data/00000-14-6ea498b5-f559-4899-bd48-5c61e7e441d5-00001.parquet        |1           |
|0      |PARQUET    |spark-warehouse/iceberg/schema/trips_mor/data/00000-2-84080325-ed0b-422a-afee-6b6df39bb8c0-0-00001.parquet       |2048        |
|0      |PARQUET    |spark-warehouse/iceberg/schema/trips_mor/data/00001-3-84080325-ed0b-422a-afee-6b6df39

Update 2: nur 1 Zeile

Iceberg packt die neue Version dieser einen Zeile in ein Mini-Datenfile

und schreibt ein Delete-File mit 1 Delete-Eintrag, der die alte Version dieser Zeile löscht


Aufräumen:

* expire_snapshots → alte Snapshots + deren nicht benötigte Dateien
aufräumen
* remove_orphan_files → Dateien löschen, die in keinem Snapshot mehr vorkommen
* rewrite_data_files → viele kleine Datafiles zu größeren zusammenfassen (Compaction)
* (optional) rewrite_position_deletes / rewrite_manifests für Feintuning

## Ducklake


In [15]:
%pip install jupysql duckdb==1.4.2 duckdb-engine
%load_ext sql
%sql duckdb://

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.3.2
    Uninstalling duckdb-1.3.2:
      Successfully uninstalled duckdb-1.3.2


Connecting to 'duckdb://'

In [16]:
%%sql
INSTALL ducklake;
LOAD ducklake;


Running query in 'duckdb://'

Success


In [17]:
%%sql
ATTACH 'ducklake:metadata.ducklake' AS my_ducklake;
USE my_ducklake;

Running query in 'duckdb://'

Success


In [18]:
%%sql
FROM ducklake_snapshots('my_ducklake');

Running query in 'duckdb://'

snapshot_id,snapshot_time,schema_version,changes,author,commit_message,commit_extra_info
0,2025-11-18 04:28:45.284766+00:00,0,{'schemas_created': ['main']},None,None,None


In [19]:
%%sql
CREATE SCHEMA IF NOT EXISTS sales;
USE sales;

Running query in 'duckdb://'

Success


In [20]:
%%sql
CREATE TABLE IF NOT EXISTS customer (
    customer_id INTEGER,  # PRIMARY KEY nicht implementiert
    last_name VARCHAR(100) NOT NULL
);
COMMIT;

Running query in 'duckdb://'

Success


In [21]:
%%sql
INSERT INTO customer (customer_id, last_name) VALUES
(1, 'Maier'),
(2, 'Schmitt'),
(3, 'Albrecht');
COMMIT;

Running query in 'duckdb://'

Success


In [22]:
%sql SELECT * FROM customer;

Running query in 'duckdb://'

customer_id,last_name
1,Maier
2,Schmitt
3,Albrecht


In [23]:
%%sql
FROM ducklake_snapshots('my_ducklake');

Running query in 'duckdb://'

snapshot_id,snapshot_time,schema_version,changes,author,commit_message,commit_extra_info
0,2025-11-18 04:28:45.284766+00:00,0,{'schemas_created': ['main']},None,None,None
1,2025-11-18 04:28:49.097244+00:00,1,{'schemas_created': ['sales']},None,None,None
2,2025-11-18 04:29:10.110254+00:00,2,{'tables_created': ['sales.customer']},None,None,None
3,2025-11-18 04:29:12.454500+00:00,2,{'tables_inserted_into': ['2']},None,None,None


In [24]:
%%sql
INSERT INTO customer (customer_id, last_name) VALUES
(4, 'Berger');
COMMIT;

Running query in 'duckdb://'

Success


In [25]:
%%sql
DELETE FROM customer WHERE customer_id = 2;
COMMIT;

Running query in 'duckdb://'

Success


In [26]:
%%sql
FROM ducklake_snapshots('my_ducklake');

Running query in 'duckdb://'

snapshot_id,snapshot_time,schema_version,changes,author,commit_message,commit_extra_info
0,2025-11-18 04:28:45.284766+00:00,0,{'schemas_created': ['main']},None,None,None
1,2025-11-18 04:28:49.097244+00:00,1,{'schemas_created': ['sales']},None,None,None
2,2025-11-18 04:29:10.110254+00:00,2,{'tables_created': ['sales.customer']},None,None,None
3,2025-11-18 04:29:12.454500+00:00,2,{'tables_inserted_into': ['2']},None,None,None
4,2025-11-18 04:29:15.225417+00:00,2,{'tables_inserted_into': ['2']},None,None,None
5,2025-11-18 04:29:42.135091+00:00,2,{'tables_deleted_from': ['2']},None,None,None


In [27]:
import duckdb

delete_file = duckdb.sql(f"""
    SELECT file FROM glob('/content/metadata.ducklake.files/sales/customer/*delete.parquet') LIMIT 1;
    """).fetchone()[0]

# Was enthält delete file?
duckdb.sql(f"""
    SELECT * FROM read_parquet('{delete_file}');
    """)


┌──────────────────────────────────────────────────────────────────────────────────────────────┬───────┐
│                                          file_path                                           │  pos  │
│                                           varchar                                            │ int64 │
├──────────────────────────────────────────────────────────────────────────────────────────────┼───────┤
│ metadata.ducklake.files/sales/customer/ducklake-019a9539-4f9c-750a-8884-67c056836de3.parquet │     1 │
└──────────────────────────────────────────────────────────────────────────────────────────────┴───────┘

Time Travel

In [28]:
%%sql
SELECT MAX(snapshot_id) FROM ducklake_snapshots('my_ducklake');

Running query in 'duckdb://'

max(snapshot_id)
5


In [29]:
%%sql
-- vor dem löschen
SELECT * FROM customer AT (VERSION => 4)

Running query in 'duckdb://'

customer_id,last_name
1,Maier
2,Schmitt
3,Albrecht
4,Berger


## Zusammenfassung: Formate im Vergleich
Die folgende Tabelle fasst **Features**, **Vorteile/Nachteile** und **Empfehlungen** zusammen.

| Format | Zentrale Features | Vorteile | Nachteile | Empfehlung |
|---|---|---|---|---|
| Delta Lake | ACID‑Transaktionen, Time‑Travel, MERGE/UPSERT, Schema‑Evolution | Reifes Spark‑Ökosystem, breite Community | Häufig Spark‑zentrisch, Log‑Pflege (VACUUM) nötig | Wenn du ohnehin Spark einsetzt und schnelle Upserts/MERGEs brauchst |
| Apache Iceberg | Snapshot‑basierte Metadaten, verborgene Partitionierung, Schema‑Evolution | Engine‑agnostisch (Spark/Trino/Flink/DuckDB), skalierbare Metadaten | Katalog/Deployment kann komplex sein | Für heterogene Engines & großes Scale-out, wenn Abfrage‑Engine frei wählbar bleiben soll |
| DuckLake | Metadaten in SQL‑DB (z. B. SQLite), Daten als Parquet, Snapshots/Changes | Sehr einfacher lokaler Katalog, schnelle Demos/POCs | Jüngeres Ökosystem, weniger Integrationen | Für einfache, portable Setups, lokale Analytics & Lehr-/Demo‑Szenarien |

In [ ]:
# Versionsübersicht

from importlib.metadata import version, PackageNotFoundError

def v(pkg):
    try:
        return version(pkg)
    except PackageNotFoundError:
        return None

rows = []

# DuckDB (Python-Package)
import duckdb
rows.append(("duckdb (py)", duckdb.__version__))

# DuckDB-Extensions (falls abrufbar)
try:
    con_chk = duckdb.connect()
    con_chk.execute("INSTALL pragma;")  # no-op, aber falls nicht vorhanden ignorieren
    exts = con_chk.execute("SELECT name, loaded, installed FROM duckdb_extensions() ORDER BY name").fetchall()
    rows.append(("duckdb extensions", ", ".join([f"{n}{'[*]' if l else ''}" for n,l,_ in exts]) or "none"))
except Exception:
    rows.append(("duckdb extensions", "n/a"))

# Delta-RS (Python-Package: 'deltalake')
dl_rs = v("deltalake")
rows.append(("deltalake (delta-rs)", dl_rs or "not installed"))

# Delta-Spark (Python-Package: 'delta-spark' / Modul: 'delta')
dl_spark = v("delta-spark") or v("delta_core") or v("delta")
rows.append(("delta-spark (Spark)", dl_spark or "not installed"))

# PySpark
ps = v("pyspark")
rows.append(("pyspark", ps or "not installed"))

# PyIceberg
pi = v("pyiceberg")
rows.append(("pyiceberg", pi or "not installed"))

# PyArrow & Pandas
pa = v("pyarrow")
pd = v("pandas")
rows.append(("pyarrow", pa or "not installed"))
rows.append(("pandas", pd or "not installed"))

# Ausgabe hübsch formatiert
w1 = max(len(k) for k,_ in rows) + 2
for k,val in rows:
    print(f"{k:<{w1}} {val}")


duckdb (py)            1.4.2
duckdb extensions      n/a
deltalake (delta-rs)   1.2.1
delta-spark (Spark)    not installed
pyspark                3.5.1
pyiceberg              0.10.0
pyarrow                18.1.0
pandas                 2.2.2
